In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Census & gmaps API Keys
from config import (census_api_key, gkey)
c = Census(census_api_key, year=2017)

# Configure gmaps
gmaps.configure(api_key=gkey)
